# Sceario Based Model Validation

This dashboard enables scenario based, end-to-end model validation streamlined for the end user.
In this case, model validation means a fully integrated, end-to-end model execution and comparison against expected results.
These comparison are based on well defined scenarios described below.

## Validation Scenarios

The scenarios used for validation can be broken down into two groups: zero cost scenarios, constraint scenarios, and cost comparison scenarios.
Both are described below.

### 1. Zero Cost Scenarios

The zero cost validation scenarios aim to validate that configuring cost driver parameters to zero will lead to zero costs in the output estimates of the models.
Each of the zero cost scenarios is described below.

1. **Zero cost per kwh**: validates that setting electricity costs to zero leads to zero electricity OpEx costs for a school.
    > **Decription:** \
    > **WHEN** the cost per kwh is set to zero for any technology,\
    > **THEN** the electricity opex costs should be zero in the model outputs.

2. **Zero cost internet mbps**: validates that setting mbps cost to zero leads to zero OpEx costs attributed to connectivity for a school.
    > **Decription:** \
    > **WHEN** the mpbs cost is set to zero for any technology,\
    > **THEN** the technology opex costs should be composed of only the maintenance costs in the output.
    
3. **Zero Technology OpEx**: validates that setting maintenance cost and mbps cost to zero leads to zero OpEx costs attributed to technology for a school.
    > **Decription:** \
    > **WHEN** the maintenance cost is set to zero for any technology,\
    > **AND** the mpbs cost is set to zero for any technology,\
    > **THEN** the technology opex costs should be zero in the output.
    
4. **Zero Intallation Cost**: validates that setting installation cost to zero leads to zero CapEx costs attributed to technology installation for a school.
    > **Decription:** \
    > **WHEN** the intallation cost is set to zero for any technology,\
    > **THEN** the technology capex cost should be composed of only the capex attributed to the service provider.
    
5. **Zero Electricity CapEx**: validates that setting solar panel install costs to zero leads to no electricity CapEx costs attributed to a school
    > **Decription:** \
    > **WHEN** the solar panel install cost is set to zero for any technology,\
    > **THEN** the electricity capex is zero.
    
### 2. Technology Constraint Scenarios

These scenarios validate technology constraints relevant to each respective model.
The validation scenarios aim to set the constraint to an extreme to drive the models to a simplified outcome.
Usually, this involved setting the constraint in a way that makes the technology an infeasible connectivity candidate.
Each constraint scenario is described below.

1. **Fiber: zero maximum connection length**: validates that setting the maximum connection length to zero leads to no schools being connected with fiber technology.
    > **Decription:** \
    > **WHEN** modeling fiber technology only,\
    > **AND** the maximum connection length is set to zero,\
    > **THEN** no schools schould be connected using fiber.

2. **Cellular: zero maximum range**: validates that setting the maximum range leads to no schools being connected with cellular technology.
    > **Decription:** \
    > **WHEN** modeling cellular technology only,\
    > **AND** the maximum range is set to zero,\
    > **THEN** no schools schould be connected using cellular.

3. **P2P: zero maximum range**: validates that setting the maximum range leads to no schools being connected with P2P technology.
    > **Decription:** \
    > **WHEN** modeling P2P technology only,\
    > **AND** the maximum range is set to zero,\
    > **THEN** no schools schould be connected using P2P.
    
4. **Demand: exceeds max bandwidth**: validates that setting the bandwidth demand above the bandwidth capacity for a given technology leads to no schools being connected with that technology.
    > **Decription:** \
    > **WHEN** the bandwidth demand exceeds the bandwidth capacity for any technology,\
    > **THEN** no schools schould be connected using that technology.
    


In [2]:
import json
from ipywidgets import FileUpload, Output, Label
import os
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.viz.notebooks.parameters.input_parameter import CategoricalDropdownParameter

import warnings
warnings.filterwarnings('ignore')


def uptade_parameters_from_config_file(inputs, file_name):
    with open(file_name, 'r') as f:
        configs = json.load(f)
    inputs.update(configs)
    return inputs


DEFINED_SCENARIOS = {'1.1: Zero Cost per kwh': {'file': 'configs/config_zero_electricity_opex.json',
                                             'scenario_name': 'zero_kwh',
                                             'type': 'full-country'},
                       '1.2: Zero Cost per mbps': {'file': 'configs/config_zero_mbps_opex.json',
                                              'scenario_name': 'zero_mbps',
                                              'type': 'full-country'},
                       '1.3: Zero Tech OpEx': {'file': 'configs/config_zero_tech_opex.json',
                                               'scenario_name': 'zero_tech_opex',
                                               'type': 'full-country'},
                       '1.4: Zero Installation Cost': {'file': 'configs/config_zero_install_cost.json',
                                                       'scenario_name': 'zero_tech_opex',
                                                       'type': 'full-country'},
                       '1.5: Zero Electricity CapEx': {'file': 'configs/config_zero_solar_capex.json',
                                                       'scenario_name': 'zero_solar_capex',
                                                       'type': 'full-country'},
                       '2.1: Fiber: zero maximum connection length': {'file': 'configs/config_constraint_fiber_zero_range.json',
                                                                      'scenario_name': 'constraint_fiber_zero_range',
                                                                      'type': 'full-country'},
                       '2.2: Cellular: zero cell tower range': {'file': 'configs/config_constraint_cellular_zero_range.json',
                                                                'scenario_name': 'constraint_cellular_zero_range',
                                                                'type': 'full-country'},
                       '2.3: P2P: zero range': {'file': 'configs/config_constraint_p2p_zero_range.json',
                                                'scenario_name': 'constraint_p2p_zero_range',
                                                'type': 'full-country'},
                       '2.4: Demand: exceeds max bandwidth': {'file': 'configs/config_constraint_bw_demand.json',
                                                              'scenario_name': 'constraint_bw_demand',
                                                              'type': 'full-country'},
                      }

options = list(DEFINED_SCENARIOS.keys())
value = options[0]
description = 'Validation Scenario:'
show_map = os.environ.get("FF_STATIC_MAP_NOTEBOOK", True)

p = CategoricalDropdownParameter(options=['Default'] + options,
                             value='Default',
                             description=description,
                             parameter_type='categorical_dropdown').parameter

inputs = CostEstimationParameterInput(local_data_workspace='workspace',show_map=show_map)

output = Output()

def on_change(change):
    with output:
        #global inputs 
        #inputs = uptade_parameters_from_config_file(inputs, DEFINED_SCENARIOS[p.value]['file'])
        file_name = DEFINED_SCENARIOS[p.value]['file']
        with open(file_name, 'r') as f:
            configs = json.load(f)
        output.clear_output()
        inputs.update(configs)
        
    #display(p, inputs.parameter_input(), output)

p.observe(on_change, names='value')
display(p, inputs.parameter_input(), output)

HTML(value='<style>\n.off-default-cell-background-color {\n    background-color: rgba(255, 227, 201, 0.95);\n …

ValueError: Connection string is either blank or malformed.

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Button, Output
from tqdm import tqdm
from IPython.display import clear_output
import sys
import time
import logging
import geopandas as gpd

from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.viz.notebooks.helpers import run_message
from giga.utils.logging import LOGGER
from giga.viz.notebooks.helpers import download_link_scenario_config
from giga.viz.notebooks.tables import display_summary_table, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, download_link_frame

def filter_schools(data_space, scenario):
    if scenario['type'] == 'select-schools':
        # filter
        return data_space.filter_schools(scenario['school_ids'])
    else:
        return data_space

button = Button(description="Run Model")
output = Output()

output_space = None
data_space = None
config = None

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        
        global config
        global data_space
        global output_space
        
        selected_scenario = DEFINED_SCENARIOS[p.value]
        
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        
        config = inputs.scenario_parameters()
        data_space = ModelDataSpace(inputs.data_parameters())
        data_space = filter_schools(data_space, selected_scenario)
        output_space = OutputSpace()
        scenario = create_scenario(config, data_space, output_space)
        
        output_space = scenario.run(progress_bar=verbose)
        run_message()
        
        display(download_link_scenario_config(inputs))
        
        display_summary_table(output_space, data_space)
        border_file = 'parameter_workspace/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp'
        borders = gpd.read_file(border_file)
        borders['NAME'] = borders['NAME'].apply(lambda x: x.lower())
        cid = inputs.data_parameters().school_data_conf.country_id
        country_id = 'rwanda' if cid == 'sample' else cid
        b = borders[borders["NAME"] == country_id]
        plot_cost_breakdown(output_space, data_space, b)
        
        table = output_to_table(output_space)
        display(download_link_frame(table))

button.on_click(on_button_clicked)

In [ ]:

import geopandas as gpd
from ipywidgets import HTML

from giga.viz.notebooks.tables import display_summary_table, output_to_capex_details, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, button_cb#, download_link_frame
from giga.data.stats.result_stats import ResultStats
from giga.viz.notebooks.components.dashboard.result_dashboard import ResultDashboard


def action():
    # Construct a results dashboard
    stats = ResultStats(data_space, output_space, inputs.all_tech_config())
    dashboard = ResultDashboard(stats, inputs)
    
    ele_map = dashboard.result_map.get_electricity_map()
    display(HTML("<hr><b><font color='#5b8ff0'>Electricity Map</b><br><br>"))
    display(ele_map)
    
    cost_map = dashboard.result_map.get_cost_map()
    display(HTML("<hr><b><font color='#5b8ff0'>Cost Map</b><br><br>"))
    display(cost_map)
    
    if output_space.fiber_costs is not None or output_space.p2p_costs is not None:
        lines_map = dashboard.result_map.get_infra_lines_map(data_map = dashboard.result_map.inputs.selected_data_map())
        display(HTML("<hr><b><font color='#5b8ff0'>Fiber/P2P Map</b><br><br>"))
        display(lines_map)
    
button, click = button_cb("Show Maps", action)
button.on_click(click)
